# 1) INSTALACIÓN E IMPORTACIÓN DE BIBLIOTECAS

In [1]:
!pip install pdfplumber
!pip install SPARQLWrapper
!pip install llama-index-embeddings-huggingface==0.1.1 sentence-transformers==2.3.1 pypdf==4.0.1 langchain==0.1.7 python-decouple==3.8 llm-templates llama-index-readers-file chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 627.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install inputimeout

In [3]:
import warnings

# Suprimir todas las advertencias
warnings.filterwarnings('ignore')

import pdfplumber
import re
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from llama_index.core import SimpleDirectoryReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from SPARQLWrapper import SPARQLWrapper, JSON
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chromadb
from chromadb.utils import embedding_functions
from chromadb import Client
from decouple import config, UndefinedValueError
from google.colab import userdata
import textwrap
from llm_templates import Formatter, Conversation
import sys
from inputimeout import inputimeout, TimeoutOccurred


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
import torch
import numpy as np





#2) SOLICITUD DE NOMBRE DE CLAVE DE HUGGING FACE COMO SECRETO DE COLAB

In [4]:
def formatear_respuesta(respuesta):
    """
    Formatea una cadena de texto para que se ajuste a un ancho específico usando textwrap.

    Parámetros:
    respuesta (str): La cadena de texto a formatear.

    Retorna:
    str: La cadena de texto formateada con el ancho especificado.
    """
    wrapped_text = textwrap.fill(respuesta, width=100)
    return wrapped_text

def obtener_clave_huggingface(max_intentos=3):
    """
    Solicita al usuario que ingrese el nombre de su clave secreta de Hugging Face almacenada en Colab.
    Verifica si la clave es correcta y devuelve el valor de la clave.
    Si el nombre de la clave no es correcto, solicita al usuario que lo ingrese nuevamente hasta un máximo de intentos.
    Si no se ingresa un nombre, detiene la ejecución del programa.

    Parámetros:
    max_intentos (int): Número máximo de intentos permitidos para ingresar el nombre correcto de la clave.

    Retorna:
    str: El valor de la clave secreta de Hugging Face si se ingresa correctamente.
    """
    intentos = 0

    while intentos < max_intentos:
        nombre_de_clave_secreta = input("Por favor, ingrese el nombre de su clave secreta de Hugging Face almacenad en Colab: ")
        print()

        if not nombre_de_clave_secreta:
            print("No se ingresó un nombre de clave. Deteniendo la ejecución.")
            print()
            sys.exit("Ejecución detenida porque no se ingresó un nombre de clave secreta de Colab correcto. Vuelva a ejecutar el programa.")

        try:
            api_key = config('HUGGINGFACE_TOKEN', userdata.get(nombre_de_clave_secreta))
            if api_key is None:
                raise UndefinedValueError
            print("Clave encontrada y cargada correctamente.")
            print()
            return api_key, nombre_de_clave_secreta

        except (UndefinedValueError, userdata.SecretNotFoundError):
            print(f"El nombre de la clave '{nombre_de_clave_secreta}' no es correcto. Por favor, ingrese el nombre correcto.")
            print()
            intentos += 1

    print(f"Se alcanzó el número máximo de intentos ({max_intentos}). Deteniendo la ejecución. Nombre de clave secreta de Colab incorrecto. Vuelva a ejecutar el programa.")
    print()
    sys.exit("Ejecución detenida porque se alcanzó el número máximo de intentos.")
    print()

# Validacón de nombre secreto de clave

solicitud_clave = formatear_respuesta("El siguiente programa requiere que usted tenga una cuenta de Hugging Face y un token generado. "
                                      "Este token además debe estar guardado como clave secretas de Colab. "
                                      "A continuación se le solicitará el nombre de clave secreta de Colab. "
                                      "Si aún no guardó su token de Hugging Face en los secretos de Colab, "
                                      "aprete ENTER, guarde el token bajo un nombre, vuelva a ejecutar el programa e ingrese el nombre secreto de token.")
print(solicitud_clave)
print()

api_key, nombre_de_clave_secreta = obtener_clave_huggingface()

# Las celdas subsiguientes pueden usar 'clave_huggingface' para autenticarse en Hugging Face
print("Nombre validado:", nombre_de_clave_secreta)


El siguiente programa requiere que usted tenga una cuenta de Hugging Face y un token generado. Este
token además debe estar guardado como clave secretas de Colab. A continuación se le solicitará el
nombre de clave secreta de Colab. Si aún no guardó su token de Hugging Face en los secretos de
Colab, aprete ENTER, guarde el token bajo un nombre, vuelva a ejecutar el programa e ingrese el
nombre secreto de token.

Por favor, ingrese el nombre de su clave secreta de Hugging Face almacenad en Colab: Token_PLN

Clave encontrada y cargada correctamente.

Nombre validado: Token_PLN


# 3) PRCESAMIENTO DE TEXTO PARA BASE DE DATOS VECTORIAL

## Funciones de procesamiento de texo

In [5]:

def obtener_enlace_bruto(url_github):
    """
    Convierte la URL de un archivo en un repositorio de GitHub desde su vista previa a una URL bruta para descarga directa.

    Esta función toma una URL que muestra un archivo en el repositorio de GitHub en vista previa (como una página de visualización)
    y la convierte en una URL que permite la descarga directa del archivo en su formato bruto.

    Parámetros:
    url_github (str): La URL del archivo en el repositorio de GitHub en vista previa (por ejemplo, `https://github.com/usuario/repositorio/blob/main/archivo.pdf`).

    Retorna:
    str: La URL bruta del archivo que se puede usar para la descarga directa (por ejemplo, `https://github.com/usuario/repositorio/raw/main/archivo.pdf`).
    """
    return url_github.replace('/blob/', '/raw/')

def extraer_texto_sin_tablas_y_graficos(url):
    """
    Extrae el texto de un archivo PDF descargado desde una URL, eliminando tablas y gráficos.

    Esta función descarga un archivo PDF desde una URL, elimina el contenido de tablas y gráficos del texto extraído, y devuelve el texto limpio.

    Parámetros:
    url (str): La URL del archivo PDF en formato bruto desde el cual se extraerá el texto.

    Retorna:
    str: El texto extraído del PDF con tablas y gráficos reemplazados por marcadores de posición.
    """
    texto = ""
    response = requests.get(url)
    with open("temp.pdf", 'wb') as temp_pdf:
        temp_pdf.write(response.content)

    with pdfplumber.open("temp.pdf") as pdf:
        for pagina in pdf.pages:
            texto_pagina = pagina.extract_text()
            tablas = pagina.extract_tables()
            if tablas:
                for tabla in tablas:
                    if tabla:
                        for fila in tabla:
                            for celda in fila:
                                if celda and celda.strip():
                                    texto_pagina = texto_pagina.replace(celda, "[TABLE]")
            for grafico in pagina.images:
                x0, y0, x1, y1 = grafico["x0"], grafico["y0"], grafico["x1"], grafico["y1"]
                palabras = pagina.extract_words()
                for palabra in palabras:
                    if (palabra["x0"] >= x0 and palabra["x1"] <= x1 and
                        palabra["top"] >= y0 and palabra["bottom"] <= y1):
                        texto_pagina = texto_pagina.replace(palabra["text"], "[IMAGE]")
            texto_pagina = texto_pagina.replace("[TABLE]", "")
            texto_pagina = texto_pagina.replace("[IMAGE]", "")
            texto += texto_pagina if texto_pagina else ""
    os.remove("temp.pdf")
    return texto

def limpiar_texto_y_eliminar_superindices_subindices(texto):
    """
    Limpia el texto eliminado superíndices y subíndices, y realiza ajustes adicionales en el formato.

    Esta función elimina caracteres de superíndices y subíndices del texto, así como otros caracteres no deseados como saltos de línea incorrectos.

    Parámetros:
    texto (str): El texto que será limpiado de superíndices, subíndices y ajustes de formato.

    Retorna:
    str: El texto limpio con superíndices y subíndices eliminados, y otros ajustes de formato aplicados.
    """
    superindices = '¹²³⁴⁵⁶⁷⁸⁹⁰⁺⁻⁼⁽⁾'
    subindices = '₀₁₂₃₄₅₆₇₈₉₊₋₌₍₎'
    texto = re.sub(f'[{superindices}{subindices}]', '', texto)
    texto = re.sub(r'-\s*\n', '', texto)
    texto = re.sub(r'(?<!\n)\n(?!\n)', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'[\n]{2,}', '\n\n', texto)
    return texto.strip()


def guardar_texto_como_txt(texto, ruta_archivo):
    """
    Guarda el texto extraído en un archivo de texto con codificación UTF-8.

    Esta función escribe el texto en un archivo de texto en la ruta especificada.

    Parámetros:
    texto (str): El texto que será guardado en el archivo.
    ruta_archivo (str): La ruta del archivo donde se guardará el texto.
    """
    with open(ruta_archivo, 'w', encoding='utf-8') as archivo:
        archivo.write(texto)

def extraer_texto_wikipedia(url):
    """
    Extrae el texto del contenido principal de un artículo de Wikipedia desde una URL.

    Esta función realiza una solicitud HTTP para obtener el contenido del artículo de Wikipedia, elimina elementos no deseados y extrae el texto de los párrafos.

    Parámetros:
    url (str): La URL del artículo de Wikipedia del cual se extraerá el texto.

    Retorna:
    str: El texto extraído del artículo de Wikipedia después de eliminar elementos no deseados.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar el contenido principal del artículo
    contenido = soup.find('div', {'class': 'mw-parser-output'})

    # Eliminar elementos no deseados (tablas, gráficos, referencias, etc.)
    for elemento in contenido(['table', 'figure', 'sup', 'span', 'img']):
        elemento.decompose()

    # Extraer texto de los párrafos del artículo
    parrafos = contenido.find_all('p')
    texto = "\n".join(parrafo.get_text() for parrafo in parrafos)

    return texto


## Procesamiento de textos

In [6]:
# Crear una carpeta en el entorno de Colab
carpeta_destino = '/content/archivos_txt_deuda_externa'
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

# Lista de archivos PDF y sus enlaces brutos en GitHub
'''
archivos_pdf = {
    "lo_interno_de_la_deuda_externa_el_caso_argentino.pdf": "https://github.com/Fran251184/prueba-/blob/1b3cdaa9171151bf267e961145889d7560074465/lo_interno_de_la_deuda_externa_el_caso_argentino.pdf",
    "la_deuda_externa_argentina.pdf": "https://github.com/Fran251184/prueba-/blob/1b3cdaa9171151bf267e961145889d7560074465/la_deuda_externa_argentina.pdf",
    "la_deuda_externa_argentina_y_la_soberanía_juridica_sus_razones_historicas.pdf": "https://github.com/Fran251184/prueba-/blob/1b3cdaa9171151bf267e961145889d7560074465/la_deuda_externa_argentina_y_la_soberanía_juridica_sus_razones_historicas.pdf",
    "la_deuda_argentina_historia,_defaul_y_reestructuracion.pdf": "https://github.com/Fran251184/prueba-/blob/1b3cdaa9171151bf267e961145889d7560074465/la_deuda_argentina_historia,_defaul_y_reestructuracion.pdf",
    "hisotira_y_evulución_de_la_deuda_argentina.pdf": "https://github.com/Fran251184/prueba-/blob/1b3cdaa9171151bf267e961145889d7560074465/hisotira_y_evulución_de_la_deuda_argentina.pdf",
    "el_rol_del_FMI_en_la_deuda_externa_argentina.pdf": "https://github.com/Fran251184/prueba-/blob/1b3cdaa9171151bf267e961145889d7560074465/el_rol_del_FMI_en_la_deuda_externa_argentina.pdf",
    "el_endeudamiento_externo_público_argentino_naturaleza_y_funciones.pdf": "https://github.com/Fran251184/prueba-/blob/1b3cdaa9171151bf267e961145889d7560074465/el_endeudamiento_externo_público_argentino_naturaleza_y_funciones.pdf"
}
'''
archivos_pdf = {
    "lo_interno_de_la_deuda_externa_el_caso_argentino.pdf": "https://github.com/Fran251184/PLN/blob/d68bb64c903d676b9352677345ae533f2818202b/PLN_TP2_Francisco_J_Alomar/lo_interno_de_la_deuda_externa_el_caso_argentino.pdf",
    "la_deuda_externa_argentina.pdf": "https://github.com/Fran251184/PLN/blob/d68bb64c903d676b9352677345ae533f2818202b/PLN_TP2_Francisco_J_Alomar/la_deuda_externa_argentina.pdf",
    "la_deuda_externa_argentina_y_la_soberanía_juridica_sus_razones_historicas.pdf": "https://github.com/Fran251184/PLN/blob/d68bb64c903d676b9352677345ae533f2818202b/PLN_TP2_Francisco_J_Alomar/la_deuda_externa_argentina_y_la_soberan%C3%ADa_juridica_sus_razones_historicas.pdf",
    "la_deuda_argentina_historia,_defaul_y_reestructuracion.pdf": "https://github.com/Fran251184/PLN/blob/d68bb64c903d676b9352677345ae533f2818202b/PLN_TP2_Francisco_J_Alomar/la_deuda_argentina_historia%2C_defaul_y_reestructuracion.pdf",
    "hisotira_y_evulución_de_la_deuda_argentina.pdf": "https://github.com/Fran251184/PLN/blob/d68bb64c903d676b9352677345ae533f2818202b/PLN_TP2_Francisco_J_Alomar/hisotira_y_evuluci%C3%B3n_de_la_deuda_argentina.pdf",
    "el_rol_del_FMI_en_la_deuda_externa_argentina.pdf": "https://github.com/Fran251184/PLN/blob/d68bb64c903d676b9352677345ae533f2818202b/PLN_TP2_Francisco_J_Alomar/el_rol_del_FMI_en_la_deuda_externa_argentina.pdf",
    "el_endeudamiento_externo_público_argentino_naturaleza_y_funciones.pdf": "https://github.com/Fran251184/PLN/blob/d68bb64c903d676b9352677345ae533f2818202b/PLN_TP2_Francisco_J_Alomar/el_endeudamiento_externo_p%C3%BAblico_argentino_naturaleza_y_funciones.pdf"
}

# Procesar cada archivo PDF
for nombre_archivo, url in archivos_pdf.items():
    enlace_bruto = obtener_enlace_bruto(url)
    texto_extraido = extraer_texto_sin_tablas_y_graficos(enlace_bruto)
    texto_limpio = limpiar_texto_y_eliminar_superindices_subindices(texto_extraido)
    ruta_txt_local = os.path.join(carpeta_destino, nombre_archivo.replace('.pdf', '.txt'))
    guardar_texto_como_txt(texto_limpio, ruta_txt_local)

    print(f"Procesado {nombre_archivo} y guardado como {ruta_txt_local}")

# URL de la página de Wikipedia
url_wikipedia = 'https://es.wikipedia.org/wiki/Historia_de_la_deuda_externa_argentina'

# Extraer y limpiar el texto de Wikipedia
texto_extraido_wikipedia = extraer_texto_wikipedia(url_wikipedia)
texto_limpio_wikipedia = limpiar_texto_y_eliminar_superindices_subindices(texto_extraido_wikipedia)
ruta_salida_wikipedia = os.path.join(carpeta_destino, 'historia_de_la_deuda_externa_argentina_wikipedia.txt')

# Guardar el texto limpio en un archivo
guardar_texto_como_txt(texto_limpio_wikipedia, ruta_salida_wikipedia)

print(f"Texto de Wikipedia extraído y guardado en {ruta_salida_wikipedia}")

Procesado lo_interno_de_la_deuda_externa_el_caso_argentino.pdf y guardado como /content/archivos_txt_deuda_externa/lo_interno_de_la_deuda_externa_el_caso_argentino.txt
Procesado la_deuda_externa_argentina.pdf y guardado como /content/archivos_txt_deuda_externa/la_deuda_externa_argentina.txt
Procesado la_deuda_externa_argentina_y_la_soberanía_juridica_sus_razones_historicas.pdf y guardado como /content/archivos_txt_deuda_externa/la_deuda_externa_argentina_y_la_soberanía_juridica_sus_razones_historicas.txt
Procesado la_deuda_argentina_historia,_defaul_y_reestructuracion.pdf y guardado como /content/archivos_txt_deuda_externa/la_deuda_argentina_historia,_defaul_y_reestructuracion.txt
Procesado hisotira_y_evulución_de_la_deuda_argentina.pdf y guardado como /content/archivos_txt_deuda_externa/hisotira_y_evulución_de_la_deuda_argentina.txt
Procesado el_rol_del_FMI_en_la_deuda_externa_argentina.pdf y guardado como /content/archivos_txt_deuda_externa/el_rol_del_FMI_en_la_deuda_externa_argentin

# 4) CHATBOOT EXPERTO EN DEUDA EXTERNA ARGENTINA

## Clasficador de embeddings

In [7]:
# Cargamos el modelo desde HuggingFace https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v2
nombre_modelo = "sentence-transformers/distiluse-base-multilingual-cased-v2"
model = SentenceTransformer(nombre_modelo)

labels = [(0, "EMB"), (1, "GRAF"), (2, "CSV")]

dataset = []

# ejemplos base de datos vectorial"
dataset.append((0, "¿Cuándo comenzó la deuda Argeninta?"))
dataset.append((0, "¿Qué gobierno fue el que más se endeudó en la historia de la deuda Argentina"))
dataset.append((0, "¿Todos los gobiernos tomaron deuda?"))
dataset.append((0, "¿Qué eventos importantes ocurrieron en la historia de la deuda externa argentina?"))
dataset.append((0, "¿Qué sabés de la deuda externa Argentina?"))
dataset.append((0, "¿Cuantos presidentes tomaron deuda?"))
dataset.append((0, "¿Cual fue la intitución multilateral de crédito que más pretó dinero al país?"))

# ejemplos base de datos de grafos"
dataset.append((1, "¿Qué es el BID?"))
dataset.append((1, "¿Qué es el Banco Interamericano de Desarrollo?"))
dataset.append((1, "¿Qué es el FMI?"))
dataset.append((1, "¿Qué es el Fondo Monetario Internacional?"))
dataset.append((1, "¿Qué es el Club de París?"))
dataset.append((1, "¿Qué es el Banco Mundial?"))


# ejemplos base de datos tabular"
dataset.append((2, "¿Cuánto era la deuda en 1980?"))
dataset.append((2, "¿Cual era el monto de la deuda en 1991?"))
dataset.append((2, "En el año 1824, ¿cuánto se endeudo el pais?"))
dataset.append((2, "En 1934, ¿cuánto sumaba la de deuda extena?"))
dataset.append((2, "En 1934, ¿cuánto sumaba la de deuda extena?"))
dataset.append((2, "¿Y en 2023?"))
dataset.append((2, "¿Sabés era la deuda externa en 1887?"))

# Preparar X e y
X = [text.lower() for label, text in dataset]
y = [label for label, text in dataset]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=3)

# Obtenemos los embeddings de BERT para los conjuntos de entrenamiento y prueba
X_train_vectorized = model.encode(X_train)
X_test_vectorized = model.encode(X_test)

# Creación y entrenamiento del modelo de Regresión Logística Multinomial
modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
modelo_LR.fit(X_train_vectorized, y_train)

# Evaluación del modelo de Regresión Logística
y_pred_LR = modelo_LR.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)

# Nuevas frases para clasificar
new_phrases = [
    "¿En 1990, cuánto había de deuda?",
    "¿Qué presidente tomó deuda?",
    "¿Sabés qué es Fondo Monetario Internacional?",
]

# Preprocesamiento y vectorización de las nuevas frases
new_phrases_lower = [text.lower() for text in new_phrases]
new_phrases_vectorized = model.encode(new_phrases_lower)

# Haciendo predicciones con el modelo entrenado
new_predictions = modelo_LR.predict(new_phrases_vectorized)

# Mostrando las predicciones junto con las frases
for text, label in zip(new_phrases, new_predictions):
    print(f"Texto: '{text}'")
    print(f"Clasificación predicha: {labels[label][1]}\n")



modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Precisión Regresión Logística: 1.0
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

Texto: '¿En 1990, cuánto había de deuda?'
Clasificación predicha: CSV

Texto: '¿Qué presidente tomó deuda?'
Clasificación predicha: EMB

Texto: '¿Sabés qué es Fondo Monetario Internacional?'
Clasificación predicha: GRAF



## 1) Funciones generales y de seteo

In [8]:
def obtener_embedding(consulta: str, modelo) -> list:
    """
    Obtiene el embedding de una consulta utilizando un modelo de embeddings.

    Parámetros:
    consulta (str): La consulta para la cual se generará el embedding.
    modelo: El modelo de embeddings utilizado para generar el embedding.

    Retorna:
    list: El embedding de la consulta como una lista.
    """
    embedding = modelo.embed_query(consulta)  # Usar el método correcto para obtener embeddings
    return embedding


def zephyr_chat_template(messages, add_generation_prompt=True):
    """
    Aplica el template de chat utilizando la librería llm-templates.

    Parámetros:
    messages (list): Lista de mensajes a incluir en el template de chat.
    add_generation_prompt (bool): Indica si se debe añadir un prompt de generación al asistente.

    Retorna:
    str: El template de chat renderizado con los mensajes proporcionados.
    """
    formatter = Formatter()
    conversation = Conversation(model='zephyr', messages=messages)
    return formatter.render(conversation, add_assistant_prompt=add_generation_prompt)


def presentarse():
    """
    Presenta al bot como un experto en la historia de la deuda externa Argentina y solicita al usuario que haga una consulta.

    Retorna:
    str: El mensaje de presentación formateado.
    """
    print("**************************************************************")
    print()
    print()
    mensaje_presentacion = (
        "Hola, soy un experto en la historia de la deuda externa Argentina. "
        "Puedo responder cualquier pregunta específica sobre el tema. "
        "¿Qué quieres consultar sobre la deuda externa Argentina? "
    )
    mensaje_presentacion = formatear_respuesta(mensaje_presentacion)
    return mensaje_presentacion



## 2) Funciones de bases de datos  



### a) Funciones de generación de base de datos vectorial

In [9]:
def mostrar_metadatos(coleccion):
    """
    Muestra los metadatos de los embeddings en una colección de Chroma.

    Parámetros:
    coleccion: La colección de Chroma de la cual se mostrarán los metadatos.
    """
    print("Metadatos de los embeddings en la colección:")
    for i in range(len(coleccion.get()['documents'])):
        metadata = coleccion.get()['metadatas'][i]
        print(f"Embedding {i}: {metadata}")

# Función para configurar y cargar Chroma
def configurar_y_cargar_chroma(fragmentos, metadatos, modelo_embedding, nombre_coleccion):
    """
    Configura y carga una colección en Chroma con los embeddings y metadatos de los fragmentos de texto.

    Parámetros:
    fragmentos (list): Lista de fragmentos de texto.
    metadatos (list): Lista de metadatos asociados a los fragmentos.
    modelo_embedding: El modelo de embeddings utilizado para generar los embeddings de los fragmentos.
    nombre_coleccion (str): El nombre de la colección a crear en Chroma.

    Retorna:
    coleccion: La colección cargada en Chroma.
    """
    cliente = chromadb.Client()

    if nombre_coleccion in [col.name for col in cliente.list_collections()]:
        cliente.delete_collection(name=nombre_coleccion)

    coleccion = cliente.create_collection(name=nombre_coleccion)

    for indice_fragmento, fragmento in enumerate(fragmentos):
        embedding = modelo_embedding.embed_query(fragmento)  # Usar el método embed_query
        metadata = metadatos[indice_fragmento]
        metadata["text"] = fragmento  # Añadir el texto al metadato
        coleccion.add(
            ids=[str(indice_fragmento)],
            documents=[fragmento],
            metadatas=[metadata],
            embeddings=[embedding]
        )
    return coleccion

### b) Función para obtener información basada en grafos de Wikidata

In [10]:
def obtener_info_wikidata(codigo):
    """
    Obtiene información de Wikidata para un código especificado.

    Parámetros:
    codigo (str): El código de Wikidata del cual se extraerá la información.

    Retorna:
    dict: Un diccionario con la etiqueta, descripción y sitio oficial del código especificado.
    """
    consulta = f"""
    SELECT ?itemLabel ?itemDescription ?officialWebsite WHERE {{
      wd:{codigo} rdfs:label ?itemLabel.
      FILTER(LANG(?itemLabel) = "es")
      wd:{codigo} schema:description ?itemDescription.
      FILTER(LANG(?itemDescription) = "es")
      wd:{codigo} wdt:P856 ?officialWebsite.
    }}
    """

    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(consulta)
    sparql.setReturnFormat(JSON)

    # Añadir cabecera User-Agent
    sparql.addCustomHttpHeader("User-Agent", "MyPythonApp/1.0 (https://example.com)")

    try:
        resultados = sparql.query().convert()
    except Exception as e:
        print(f"Error al consultar Wikidata: {e}")
        return {
            "Label": None,
            "Description": None,
            "Official Website": None
        }

    if resultados["results"]["bindings"]:
        resultado = resultados["results"]["bindings"][0]
        etiqueta = resultado['itemLabel']['value']
        descripcion = resultado['itemDescription']['value']
        sitio_oficial = resultado['officialWebsite']['value']
        return {
            "Label": etiqueta,
            "Description": descripcion,
            "Official Website": sitio_oficial
        }
    else:
        return {
            "Label": None,
            "Description": None,
            "Official Website": None
        }


### c) Función de obtención de información de base de datos tabular (csv)

In [11]:
def buscar_deuda_anual(año: int = None, df: pd.DataFrame = None) -> tuple:
    """
    Busca la deuda anual de Argentina en un año específico dentro de un DataFrame.

    Parámetros:
    año (int, opcional): El año para el cual se buscará la deuda. Si no se proporciona, se utilizará el año por defecto (None).
    df (pd.DataFrame): El DataFrame que contiene los datos de deuda.

    Retorna:
    tuple: Una tupla que contiene la deuda encontrada en el año especificado o un mensaje de que no se encontró información,
           y un booleano que indica si se encontró o no la información.
    """
    bandera = False
    if año is None:
        return ("No se proporcionó un año específico para la búsqueda de deuda. El registro disponible comienza en el año 1824 y finaliza en el año 2023, pero carece de datos entre los años 1825 y 1863.", bandera)

    deuda = df.loc[df['anio'] == año, 'monto_usd'].values
    if deuda.size > 0:
        bandera = True
        return (f"El monto de la deuda externa total Argentina en el año {año} era {deuda[0]} millones de USD.", bandera)
    else:
        return (f"No se encontró información sobre la deuda en el año {año}.", bandera)


## 3) Funciones de prompt y respuestas de clasificación

### a)Clasificador de preguntas relasionadas a la deuda externa

In [12]:
def clasificar_consulta(consulta: str) -> str:
    """
    Clasifica una consulta de usuario según una plantilla predefinida.

    Parámetros:
    consulta (str): La consulta del usuario a clasificar.

    Retorna:
    str: La clasificación de la consulta (CSV, GRAF, EMB o OTRO).
    """
    PLANTILLA_CLASIFICACION_BASE_DATOS = (
        "Dadas las siguientes categorías, clasifica el texto en únicamente una categoría.\n"
        "Categorías:\n"
        "EMB: preguntas históricas relacionadas con la deuda externa.\n"
        "CSV: montos de deuda por año (ejemplo 1500).\n"
        "GRAF: definición y función actual de las siguientes instituciones: Fondo Monetario Internacional (FMI), Banco Mundial, Club de París y Banco Interamericano de Desarrollo (BID).\n"
        "OTRO: ninguna de las anteriores.\n\n"
        "Ejemplos:\n"
        "Pregunta: ¿Cuándo comenzó la deuda Argentina?\n"
        "Clasificación: EMB\n\n"
        "Pregunta: ¿Qué eventos importantes ocurrieron en la historia de la deuda externa argentina?\n"
        "Clasificación: EMB\n\n"
        "Pregunta: ¿Cuál fue la institución multilateral de crédito que más prestó dinero al país?\n"
        "Clasificación: EMB\n\n"
        "Pregunta: ¿Qué es el BID?\n"
        "Clasificación: GRAF\n\n"
        "Pregunta: ¿Qué es el Banco Interamericano de Desarrollo?\n"
        "Clasificación: GRAF\n\n"
        "Pregunta: ¿Qué es el FMI?\n"
        "Clasificación: GRAF\n\n"
        "Pregunta: ¿Qué es el Fondo Monetario Internacional?\n"
        "Clasificación: GRAF\n\n"
        "Pregunta: ¿Qué es el Club de París?\n"
        "Clasificación: GRAF\n\n"
        "Pregunta: ¿Qué es el Banco Mundial?\n"
        "Clasificación: GRAF\n\n"
        "Pregunta: ¿qué es el bid?\n"
        "Clasificación: GRAF\n\n"
        "Pregunta: ¿qué es el banco interamericano de desarrollo?\n"
        "Clasificación: GRAF\n\n"
        "Pregunta: ¿qué es el fmi?\n"
        "Clasificación: GRAF\n\n"
        "Pregunta: ¿qué es el fondo monetario internacional?\n"
        "Clasificación: GRAF\n\n"
        "Pregunta: ¿cuánto era la deuda en 1980?\n"
        "Clasificación: CSV\n\n"
        "Pregunta: ¿Cuál era el monto de la deuda en 1991?\n"
        "Clasificación: CSV\n\n"
        "Pregunta: En el año 1824, ¿cuánto se endeudó el país?\n"
        "Clasificación: CSV\n\n"
        "Pregunta: ¿Cuándo comenzó la deuda Argentina?\n"
        "Clasificación: EMB\n\n"
        "Pregunta: ¿Qué gobierno fue el que más se endeudó en la historia de la deuda Argentina?\n"
        "Clasificación: EMB\n\n"
        "Pregunta: ¿Todos los gobiernos tomaron deuda?\n"
        "Clasificación: EMB\n\n"
        "Pregunta: ¿Qué eventos importantes ocurrieron en la historia de la deuda externa argentina?\n"
        "Clasificación: EMB\n\n"
        "Pregunta: ¿Qué sabés de la deuda externa Argentina?\n"
        "Clasificación: EMB\n\n"
        "Pregunta: ¿Cuántos presidentes tomaron deuda?\n"
        "Clasificación: EMB\n\n"
        "Pregunta: ¿Cuál fue la institución multilateral de crédito que más prestó dinero al país?\n"
        "Clasificación: EMB\n\n"
        "Pregunta: ¿Quién fue Juan Domingo Perón?\n"
        "Clasificación: EMB\n\n"
        "Pregunta: ¿Qué fue el préstamo Brading?\n"
        "Clasificación: EMB\n\n"
        "Pregunta: {consulta}\n"
        "Clasificación: "
    )

    prompt = PLANTILLA_CLASIFICACION_BASE_DATOS.format(consulta=consulta)
    clasificacion = generar_clasificacion(prompt)
    print(f"Salida del clasificador: '{clasificacion}'")
    return clasificacion.strip().upper()

def generar_clasificacion(prompt: str) -> str:
    """
    Genera la clasificación de una consulta utilizando el modelo de Hugging Face.

    Parámetros:
    prompt (str): El prompt con la consulta a clasificar.

    Retorna:
    str: La clasificación generada por el modelo.
    """
    try:
        api_key = config('HUGGINGFACE_TOKEN', default=userdata.get('Token_PLN'))
        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"
        headers = {"Authorization": f"Bearer {api_key}"}
        data = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 4,
                "temperature": 0.2,
                "top_k": 1,
                "top_p": 0.1
            }
        }
        response = requests.post(api_url, headers=headers, json=data)
        response.raise_for_status()

        respuesta = response.json()[0]["generated_text"]
        clasificacion = respuesta[len(prompt):].strip()
        return clasificacion
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return "OTRO"


### b) Clasificador de continuidad de conversación

In [13]:
def clasificador_seguimiento_conversacion(consulta: str) -> int:
    """
    Clasifica una consulta de usuario como saludo, despedida o otra consulta.

    Parámetros:
    consulta (str): La consulta del usuario a clasificar.

    Retorna:
    int: 1 si es un saludo, 2 si es una despedida, 3 si es otra consulta.
    """
    PLANTILLA_CLASIFICACION_SALUDO_DESPEDIDA = (
        "Dadas las siguientes categorías, clasifica el texto en únicamente una categoría.\n"
        "Categorías:\n"
        "1: saludo.\n"
        "2: despedida.\n"
        "3: otra consulta.\n\n"
        "Ejemplos:\n"
        "Consulta: Hola, ¿cómo estás?\n"
        "Clasificación: 1\n\n"
        "Consulta: Buenos días\n"
        "Clasificación: 1\n\n"
        "Consulta: Adiós\n"
        "Clasificación: 2\n\n"
        "Consulta: Hasta luego\n"
        "Clasificación: 2\n\n"
        "Consulta: ¿Cuál es el monto de la deuda en 1991?\n"
        "Clasificación: 3\n\n"
        "Consulta: ¿Qué es el FMI?\n"
        "Clasificación: 3\n\n"
        "Consulta: {consulta}\n"
        "Clasificación: "
    )

    prompt = PLANTILLA_CLASIFICACION_SALUDO_DESPEDIDA.format(consulta=consulta)
    clasificacion = generar_respuesta_clasificador_seguimiento(prompt)
    print(f"Salida del clasificador: '{clasificacion}'")
    return int(clasificacion.strip())

def generar_respuesta_clasificador_seguimiento(prompt: str) -> str:
    """
    Genera la clasificación de una consulta utilizando el modelo de Hugging Face.

    Parámetros:
    prompt (str): El prompt con la consulta a clasificar.

    Retorna:
    str: La clasificación generada por el modelo.
    """
    try:
        api_key = config('HUGGINGFACE_TOKEN', default=userdata.get('Token_PLN'))
        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"
        headers = {"Authorization": f"Bearer {api_key}"}
        data = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 1,  # Generar un solo token para la clasificación
                "temperature": 0.1,   # Hacer la generación lo más determinística posible
                "top_k": 1,           # Usar el token con la mayor probabilidad
                "top_p": 0.98
            }
        }
        response = requests.post(api_url, headers=headers, json=data)
        response.raise_for_status()

        respuesta = response.json()[0]["generated_text"]
        clasificacion = respuesta[len(prompt):].strip()
        return clasificacion
    except Exception as e:
        print(f"Error en la clasificación: {e}")
        return "3"  # Retorna 3 (otra consulta) en caso de error


## 4) Funciones de promt y respuesta a preguntas

### a) Sobre la deduda externa Argentina

In [14]:

def preparar_prompt(consulta_str: str, contexto_str: str) -> str:
    """
    Prepara el prompt en estilo QA utilizando una plantilla predefinida.

    Parámetros:
    consulta_str (str): La consulta del usuario.
    contexto_str (str): El contexto de información proporcionado para responder la consulta.

    Retorna:
    str: El prompt final preparado para ser enviado al modelo.
    """
    PLANTILLA_QA = (
        "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos.\n\n"
        "Importante: Tus respuestas deben ser breves, específicas y concluyentes.\n\n"
        "Responde siempre en español y proporciona una respuesta directa basada únicamente en la información proporcionada en el contexto.\n\n"
        "Formula la respuesta sin hacer referencia explícita al contexto.\n\n"
        "Pregunta: {consulta_str}\n\n"
        "Contexto:\n{contexto_str}\n\n"
        "Respuesta: "
    )

    mensajes = [
        {"role": "system", "content": "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos. "
         "Importante: Tus respuestas deben ser breves, específicas y concluyentes "
         "Responde siempre en español y proporciona una respuesta directa basada únicamente en la información proporcionada en el contexto. "
         "Formula la respuesta sin hacer referencia explícita al contexto."},
        {"role": "user", "content": PLANTILLA_QA.format(contexto_str=contexto_str, consulta_str=consulta_str)}
    ]

    prompt_final = zephyr_chat_template(mensajes)
    return prompt_final

def preparar_prompt_grafos(consulta_str: str, contexto_dict: dict) -> str:
    """
    Prepara el prompt en estilo QA específico para consultas sobre grafos, utilizando una plantilla predefinida.

    Parámetros:
    consulta_str (str): La consulta del usuario.
    contexto_dict (dict): El contexto de información proporcionado para responder la consulta, incluyendo la descripción y el enlace oficial.

    Retorna:
    str: El prompt final preparado para ser enviado al modelo.
    """
    PLANTILLA_QA_GRAFOS = (
        "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos.\n\n"
        "Responde siempre en español. Usa exclusivamente la información proporcionada en el contexto para responder la pregunta.\n\n"
        "Importante: incluye siempre el link del sitio web en la respuesta. Formula la respuesta sin hacer referencia explícita al contexto.\n\n"
        "La información de contexto es la siguiente:\n"
        "---------------------\n"
        "Descripción: {description}\n"
        "Página oficial: {official_website}\n"
        "---------------------\n"
        "Responde la siguiente pregunta utilizando únicamente el contexto proporcionado.\n"
        "Pregunta: {consulta_str}\n"
        "Respuesta: "
    )

    contexto_str = PLANTILLA_QA_GRAFOS.format(
        description=contexto_dict['Description'],
        official_website=contexto_dict['Official Website'],
        consulta_str=consulta_str
    )

    mensajes = [
        {"role": "system", "content": "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos."
                                      "Responde siempre en español. Usa exclusivamente la información proporcionada en el contexto para "
                                      "responder la pregunta. Incluye siempre el link del sitio web en la respuesta. Formula la respuesta sin hacer referencia explícita al contexto."},
        {"role": "user", "content": contexto_str}
    ]

    prompt_final = zephyr_chat_template(mensajes)
    return prompt_final



def generar_respuesta(prompt: str, max_new_tokens: int = 250) -> str:
    """
    Genera una respuesta para una consulta del usuario utilizando el modelo de Hugging Face.

    Parámetros:
    prompt (str): El prompt con la consulta del usuario.
    max_new_tokens (int): Número máximo de nuevos tokens a generar en la respuesta.

    Retorna:
    str: La respuesta generada por el modelo.
    """
    try:
        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"
        headers = {"Authorization": f"Bearer {api_key}"}
        data = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": max_new_tokens,  # Limita la longitud de la respuesta
                "temperature": 0.3,  # Controla la aleatoriedad de la generación
                "top_k": 10,  # Limita a las 5 palabras más probables en cada paso
                "top_p": 0.96,  # Usa palabras cuya probabilidad acumulada esté por debajo de 0.9
                "stop_sequences": ["."]  # Detiene la generación si se encuentra un punto
            }
        }
        response = requests.post(api_url, headers=headers, json=data)
        if response.status_code != 200:
            raise Exception(f"Request failed with status code {response.status_code}. Response: {response.text}")

        respuesta = response.json()[0]["generated_text"][len(prompt):].strip()

        return respuesta
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Hubo un error al generar la respuesta."

def limpiar_respuesta(respuesta: str) -> str:
    """
    Limpia una respuesta verificando si termina en un punto.
    Si no termina en punto, elimina todo lo que está después del último punto en el string.

    Parámetros:
    respuesta (str): La respuesta a limpiar.

    Retorna:
    str: La respuesta limpia que termina en punto o el string original si no hay puntos.
    """
    # Verificar si la respuesta ya termina en punto
    if respuesta.endswith('.'):
        return respuesta

    # Encontrar la posición del último punto en el string
    ultimo_punto = respuesta.rfind('.')

    # Si no hay un punto en la respuesta, retornar el string original
    if ultimo_punto == -1:
        return respuesta

    # Retornar la respuesta hasta el último punto
    return respuesta[:ultimo_punto + 1]




###b) Sobre continuidad de convesación ("humanización)

In [15]:
def preparar_prompt_saludo(consulta_str: str) -> str:
    """
    Prepara el prompt en estilo QA para saludar al usuario y recordar la función del asistente.

    Parámetros:
    consulta_str (str): La consulta del usuario.

    Retorna:
    str: El prompt final preparado para ser enviado al modelo.
    """
    mensajes = [
        {"role": "system", "content": "Saluda amablemente al usuario y menciona que eres un experto en la deuda externa de Argentina. Responde de modo breve."},
        {"role": "user", "content": consulta_str}
    ]

    prompt_final = zephyr_chat_template(mensajes)
    return prompt_final

def preparar_prompt_despedida(consulta_str: str) -> str:
    """
    Prepara el prompt en estilo QA para despedirse del usuario y agradecerle.

    Parámetros:
    consulta_str (str): La consulta del usuario.

    Retorna:
    str: El prompt final preparado para ser enviado al modelo.
    """
    mensajes = [
        {"role": "system", "content": "Despídete amablemente del usuario y agradécele por su consulta. Responde de modo breve."},
        {"role": "user", "content": consulta_str}
    ]

    prompt_final = zephyr_chat_template(mensajes)
    return prompt_final

def prompt_de_seguimiento_conversacion(consulta_str: str) -> str:
    """
    Prepara el prompt en estilo QA para indicar al usuario que no se ha comprendido su pregunta y sugerir que intente de otro modo o haga otra pregunta.

    Parámetros:
    consulta_str (str): La consulta del usuario.

    Retorna:
    str: El prompt final preparado para ser enviado al modelo.
    """
    mensajes = [
        {"role": "system", "content": "Pide disculpas por no poder responder. Eres experto en la deuda externa de Argentina. Sugerir al usuario que reformule la pregunta y que sea específica sobre la deuda externa de Argentina. Responde de modo breve."},
        {"role": "user", "content": ""}
    ]

    prompt_final = zephyr_chat_template(mensajes)
    return prompt_final

def generar_respuesta_continuidad(prompt: str, max_new_tokens: int = 20) -> str:
    """
    Genera una respuesta para una consulta del usuario utilizando el modelo de Hugging Face.

    Parámetros:
    prompt (str): El prompt con la consulta del usuario.
    max_new_tokens (int): Número máximo de nuevos tokens a generar en la respuesta.

    Retorna:
    str: La respuesta generada por el modelo.
    """
    try:
        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"
        headers = {"Authorization": f"Bearer {api_key}"}
        data = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": 0.1,
                "top_k": 10,
                "top_p": 0.96,
                "stop_sequences": [".", "\n"]
            }
        }
        response = requests.post(api_url, headers=headers, json=data)
        response.raise_for_status()

        respuesta = response.json()[0]["generated_text"][len(prompt):].strip()
        return respuesta
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Hubo un error al generar la respuesta."

## 5) Función de procesamiento de consultas

In [16]:

def procesar_consulta(consulta_usuario, df, modelo_embedding_1, coleccion):
    """
    Procesa una consulta del usuario, clasifica la consulta y genera una respuesta adecuada.

    Parámetros:
    consulta_usuario (str): La consulta del usuario.
    df (pd.DataFrame): El DataFrame que contiene los datos de deuda.
    modelo_embedding_1: El modelo de embeddings utilizado para generar embeddings de la consulta.
    coleccion: La colección de Chroma que contiene los embeddings de los fragmentos de texto.

    Retorna:
    str: La respuesta generada para la consulta del usuario.
    """
    print("####### PROCESAMIENTO (EL USUARIO NO TENDRÍA ACCESO) #########################################")
    print()
    print('Clasificando la consulta...\n')

    flag = True
    clasificacion = clasificar_consulta(consulta_usuario)
    embedding_consulta = obtener_embedding(consulta_usuario, modelo_embedding_1)

    if clasificacion == "CSV":
        print("Clasificación es CSV. Extrayendo año y buscando en el CSV.")
        coincidencia_año = re.search(r'\b\d{1,4}\b', consulta_usuario)

        if coincidencia_año:
            año = int(coincidencia_año.group())
            respuesta_df, bandera = buscar_deuda_anual(año, df)
            if bandera:
                prompt_final = preparar_prompt(consulta_usuario, respuesta_df)
                print('-------------------------------------------------------')
                print(f'Prompt enviado al modelo (debug): {prompt_final}')
                print('-------------------------------------------------------')
                print("####### PROCESAMIENTO (EL USUARIO NO TENDRÍA ACCESO) #########################################")
                print()
                print()
                respuesta = generar_respuesta(prompt_final)
            else:
                print("####### PROCESAMIENTO (EL USUARIO NO TENDRÍA ACCESO) #########################################")
                respuesta = "Si quiere saber sobre el monto de la deuda para un año, debe ingresar un dígito de cuatro cifras. El registro disponible comienza en el año 1824 y finaliza en el año 2023, pero carece de datos entre los años 1825 y 1863."
                print()


        else:

            print("####### PROCESAMIENTO (EL USUARIO NO TENDRÍA ACCESO) #########################################")
            respuesta = "Si quiere saber sobre el monto de la deuda para un año, debe ingresar un dígito de cuatro cifras. El registro disponible comienza en el año 1824 y finaliza en el año 2023, pero carece de datos entre los años 1825 y 1863."
            print()

    elif clasificacion == "EMB":
        print("Clasificación es EMB. Extrayendo info de base de datos vectorial.")
        print()
        # Recuperar documentos similares desde Chroma
        contexto_str = ''
        documentos_separados = []


        resultados = coleccion.query(query_embeddings=[embedding_consulta], n_results=4)
        for doc in resultados['documents'][0]:
            documentos_separados.append(doc)
            contexto_str += f"{doc}\n"

        # Imprimir documentos separados
        for i, doc in enumerate(documentos_separados, 1):
            print(f'Texto de embedding {i}: {doc}\n')
        prompt_final = preparar_prompt(consulta_usuario, contexto_str)

        print('-------------------------------------------------------')
        print(f'Prompt enviado al modelo (debug): {prompt_final}')
        print('-------------------------------------------------------')
        print("####### PROCESAMIENTO (EL USUARIO NO TENDRÍA ACCESO) #########################################")
        print()
        print()
        respuesta = generar_respuesta(prompt_final)
        respuesta = limpiar_respuesta(respuesta)

    elif clasificacion == "GRAF":
        print("Clasificación es GRAF. Extrayendo info de wikidata.")
        similitudes = {
            "Q7804": cosine_similarity([embedding_consulta], [embedding_fmi])[0][0],
            "Q1153087": cosine_similarity([embedding_consulta], [embedding_bid])[0][0],
            "Q461736": cosine_similarity([embedding_consulta], [embedding_cp])[0][0],
            "Q7164": cosine_similarity([embedding_consulta], [embedding_bm])[0][0]
        }

        codigo_max_similitud = max(similitudes, key=similitudes.get)
        print(f'La mayor similitud es con el código: {codigo_max_similitud}')
        print(f'Similitudes: {similitudes}')

        respuesta_wikidata = obtener_info_wikidata(codigo_max_similitud)
        prompt_final = preparar_prompt_grafos(consulta_usuario, respuesta_wikidata)
        print('-------------------------------------------------------')
        print(f'Prompt enviado al modelo (debug): {prompt_final}')
        print('-------------------------------------------------------')
        print("####### PROCESAMIENTO (EL USUARIO NO TENDRÍA ACCESO) #########################################")
        print()
        print()
        respuesta = generar_respuesta(prompt_final)

    else:
        print("Clasificación de continuidad de conversación...")
        print()

        clasificacion = clasificador_seguimiento_conversacion(consulta_usuario)

        if clasificacion == 1:
            prompt_final = preparar_prompt_saludo(consulta_usuario)

            print('-------------------------------------------------------')
            print(f'Prompt enviado al modelo (debug): {prompt_final}')
            print('-------------------------------------------------------')
            print("####### PROCESAMIENTO (EL USUARIO NO TENDRÍA ACCESO) #########################################")
            print()
            print()

            respuesta = generar_respuesta_continuidad(prompt_final)
            respuesta = limpiar_respuesta(respuesta)

        elif clasificacion == 2:
            prompt_final = preparar_prompt_despedida(consulta_usuario)

            print('-------------------------------------------------------')
            print(f'Prompt enviado al modelo (debug): {prompt_final}')
            print('-------------------------------------------------------')
            print("####### PROCESAMIENTO (EL USUARIO NO TENDRÍA ACCESO) #########################################")
            print()
            print()

            respuesta = generar_respuesta_continuidad(prompt_final)
            respuesta = limpiar_respuesta(respuesta)
            flag = False

        else:

            prompt_final = prompt_de_seguimiento_conversacion(consulta_usuario)

            print('-------------------------------------------------------')
            print(f'Prompt enviado al modelo (debug): {prompt_final}')
            print('-------------------------------------------------------')
            print("####### PROCESAMIENTO (EL USUARIO NO TENDRÍA ACCESO) #########################################")
            print()
            print()
            respuesta = generar_respuesta_continuidad(prompt_final)
            respuesta = limpiar_respuesta(respuesta)

        respuesta = generar_respuesta(prompt_final)
        respuesta = limpiar_respuesta(respuesta)
    return respuesta, flag


## 6) Código de ejecución del programa

In [17]:
# Inicializar el modelo de embeddings y el índice de Chroma
print('Cargando modelo de embeddings...')
nombre_modelo = "sentence-transformers/distiluse-base-multilingual-cased-v2"
embed_model = HuggingFaceEmbeddings(model_name=nombre_modelo)

# URL del archivo CSV en el repositorio de GitHub
url_csv = "https://raw.githubusercontent.com/Fran251184/PLN/main/PLN_TP2_Francisco_J_Alomar/deuda_externa_argentina_1824_2023_actualizado.csv"
# Cargar el archivo CSV desde la URL
df = pd.read_csv(url_csv)

# Cargar los documentos de la carpeta
documentos = SimpleDirectoryReader(input_dir='/content/archivos_txt_deuda_externa/').load_data()

# Dividir los textos en fragmentos usando Langchain
print('Dividiendo los documentos en fragmentos...')
divisor_texto = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", " "])

# Inicializar lista para almacenar todos los fragmentos y sus metadatos
todos_los_fragmentos = []
todos_los_metadatos = []

##########################SECCIÓN DE CÓDIGO QUE HACE EMBEDDING DE UN SOLO DOCMENTO###############
## Tomar solo el primer documento de la lista
#documento = documentos[0]
#
## Dividir el texto del documento en fragmentos
#fragmentos = divisor_texto.split_text(documento.text)
#nombre_archivo = os.path.basename(documento.metadata["file_path"])  # Obtener el nombre del archivo
#
## Iterar sobre los fragmentos del documento
#for i, fragmento in enumerate(fragmentos):
#    todos_los_fragmentos.append(fragmento)
#    todos_los_metadatos.append({"chunk_index": i, "source": nombre_archivo})
#
##########################SECCIÓN DE CÓDIGO QUE HACE EMBEDDING DE UN SOLO DOCMENTO###############


# Iterar sobre todos los documentos y dividirlos en fragmentos
for documento in documentos:
    fragmentos = divisor_texto.split_text(documento.text)
    nombre_archivo = os.path.basename(documento.metadata["file_path"])  # Obtener el nombre del archivo
    for i, fragmento in enumerate(fragmentos):
        todos_los_fragmentos.append(fragmento)
        todos_los_metadatos.append({"chunk_index": i, "source": nombre_archivo})

# Configurar y cargar Chroma
coleccion = configurar_y_cargar_chroma(todos_los_fragmentos, todos_los_metadatos, embed_model, 'coleccion_documentos')
print("Chroma cargado con embeddings y metadatos...")

# Mostrar los metadatos
#mostrar_metadatos(coleccion)

# Obtener embeddings de nombre de instituciones internacionales de crédito
embedding_fmi = obtener_embedding("Fondo Monetario Internacional, fmi, FMI", embed_model)
embedding_bid = obtener_embedding("Banco Interamericano de Desarrollo, bid, BID", embed_model)
embedding_cp = obtener_embedding("Club de París", embed_model)
embedding_bm = obtener_embedding("Banco Mundial", embed_model)

# Interacción con el usuario
print(presentarse())
print()
flag = True

while flag:

    consulta_usuario = input("Tu pregunta: ")
    print()
    respuesta, flag = procesar_consulta(consulta_usuario, df, embed_model, coleccion)
    respuesta_formateada = formatear_respuesta(respuesta)

    print("Respuesta:")
    print()
    print(respuesta_formateada)
    print()

    if not flag:
        break





Cargando modelo de embeddings...
Dividiendo los documentos en fragmentos...
Chroma cargado con embeddings y metadatos...
**************************************************************


Hola, soy un experto en la historia de la deuda externa Argentina. Puedo responder cualquier
pregunta específica sobre el tema. ¿Qué quieres consultar sobre la deuda externa Argentina?

Tu pregunta: Hola, cómo estás? 

####### PROCESAMIENTO (EL USUARIO NO TENDRÍA ACCESO) #########################################

Clasificando la consulta...

Salida del clasificador: 'ÓTRO'
Clasificación de continuidad de conversación...

Salida del clasificador: '1'
-------------------------------------------------------
Prompt enviado al modelo (debug): <|system|>Saluda amablemente al usuario y menciona que eres un experto en la deuda externa de Argentina. Responde de modo breve.</s>
<|user|>Hola, cómo estás? </s>
<|assistant|>

-------------------------------------------------------
####### PROCESAMIENTO (EL USUARIO N